Import the data from Edge Impulse. You can obtain the URL from the Dashboard, right-click on the download icon next to 'Spectral features data' and 'Spectral features labels', and click **Copy link location**.

In [1]:
import numpy as np
import requests

API_KEY = 'ei_382beb9b57c1c047ee19a04277381328b8760d36c3805e5d84856a52c4f19311'

X = (requests.get('https://studio.edgeimpulse.com/v1/api/4286/training/5/x', headers={'x-api-key': API_KEY})).content
Y = (requests.get('https://studio.edgeimpulse.com/v1/api/4286/training/5/y', headers={'x-api-key': API_KEY})).content

Store the data in a temporary file, and load it back through Numpy.

In [2]:
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]

Define our labels and split the data up in a test and training set:

In [3]:
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "Bronchiectasis", "Bronchiolitis", "COPD", "Healthy", "Pneumonia", "URTI" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def set_batch_size(batch_size, train_dataset, validation_dataset):
    shuffle_buffer_size = batch_size * 3
    train_dataset = train_dataset.shuffle(shuffle_buffer_size).batch(batch_size, drop_remainder=True).prefetch(1)
    validation_dataset = validation_dataset.shuffle(shuffle_buffer_size).batch(batch_size, drop_remainder=True).prefetch(1)
    return train_dataset, validation_dataset

callbacks = []


/home/antpc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/antpc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/antpc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/antpc/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

Train the model:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten, Reshape, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import MaxNorm
# model architecture
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1], ), name='x_input'))
model.add(Reshape((int(X_train.shape[1] / 13), 13, 1), input_shape=(X_train.shape[1], )))
model.add(Conv2D(10, kernel_size=5, activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
model.add(AveragePooling2D(pool_size=2, padding='same'))
model.add(Conv2D(5, kernel_size=5, activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
model.add(AveragePooling2D(pool_size=2, padding='same'))
model.add(Flatten())
model.add(Dense(classes, activation='softmax', name='y_pred', kernel_constraint=MaxNorm(3)))
# this controls the learning rate
opt = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999)
# train the neural network
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=2, epochs=200, validation_data=(X_test, Y_test), verbose=2)

Train on 4112 samples, validate on 1029 samples
Epoch 1/200
4112/4112 - 4s - loss: 1.6885 - accuracy: 0.2782 - val_loss: 1.6816 - val_accuracy: 0.2877
Epoch 2/200
4112/4112 - 4s - loss: 1.6671 - accuracy: 0.2806 - val_loss: 1.6786 - val_accuracy: 0.3090
Epoch 3/200
4112/4112 - 4s - loss: 1.6618 - accuracy: 0.2879 - val_loss: 1.6791 - val_accuracy: 0.3061
Epoch 4/200
4112/4112 - 4s - loss: 1.6569 - accuracy: 0.2913 - val_loss: 1.6775 - val_accuracy: 0.3052
Epoch 5/200
4112/4112 - 4s - loss: 1.6516 - accuracy: 0.3057 - val_loss: 1.6711 - val_accuracy: 0.3120
Epoch 6/200
4112/4112 - 4s - loss: 1.6466 - accuracy: 0.3074 - val_loss: 1.6680 - val_accuracy: 0.3226
Epoch 7/200
4112/4112 - 4s - loss: 1.6410 - accuracy: 0.3157 - val_loss: 1.6671 - val_accuracy: 0.3110
Epoch 8/200
4112/4112 - 4s - loss: 1.6344 - accuracy: 0.3232 - val_loss: 1.6654 - val_accuracy: 0.3100
Epoch 9/200
4112/4112 - 4s - loss: 1.6269 - accuracy: 0.3276 - val_loss: 1.6526 - val_accuracy: 0.3362
Epoch 10/200
4112/4112 - 

Epoch 80/200
4112/4112 - 4s - loss: 1.2917 - accuracy: 0.4990 - val_loss: 1.4783 - val_accuracy: 0.4130
Epoch 81/200
4112/4112 - 4s - loss: 1.2887 - accuracy: 0.4988 - val_loss: 1.4820 - val_accuracy: 0.4082
Epoch 82/200
4112/4112 - 4s - loss: 1.2849 - accuracy: 0.5029 - val_loss: 1.4721 - val_accuracy: 0.4091
Epoch 83/200
4112/4112 - 4s - loss: 1.2823 - accuracy: 0.4956 - val_loss: 1.5007 - val_accuracy: 0.3916
Epoch 84/200
4112/4112 - 4s - loss: 1.2785 - accuracy: 0.5002 - val_loss: 1.4944 - val_accuracy: 0.4023
Epoch 85/200
4112/4112 - 4s - loss: 1.2754 - accuracy: 0.5058 - val_loss: 1.4615 - val_accuracy: 0.4072
Epoch 86/200
4112/4112 - 4s - loss: 1.2731 - accuracy: 0.5073 - val_loss: 1.4618 - val_accuracy: 0.4052
Epoch 87/200
4112/4112 - 4s - loss: 1.2722 - accuracy: 0.5102 - val_loss: 1.4638 - val_accuracy: 0.4091
Epoch 88/200
4112/4112 - 4s - loss: 1.2686 - accuracy: 0.5092 - val_loss: 1.4661 - val_accuracy: 0.4159
Epoch 89/200
4112/4112 - 4s - loss: 1.2684 - accuracy: 0.5119 - 

In [5]:
# Save the model to disk
model.save('saved_model')